In [ ]:
## For subsidence modeling in extensional basins - synthetic test case

## --- Load required pacages, install SubsidenceChron if required
# Load (and install if necessary) the Chron.jl package
    
    using SubsidenceChron
    using StatGeochem, Distributions, Plots, Statistics, StatsBase

In [ ]:
## --- Part 1: Decompaction and Backstripping

    # # # # # # # # # # # Enter stratigraphic information here! # # # # # # # # # # # #
    # Import the data file (.csv)
    data_csv = importdataset("Input/McCoyShort/McCoy_highres_short2.csv",',')
    # Obtain stratigraphic info from the data file
    nLayers = length(data_csv["Thickness"])
    strat = NewStratData(nLayers)
    strat.Lithology          = data_csv["Lithology"]
    strat.Thickness         .= data_csv["Thickness"]
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #


    # # # # # # # # OPTIONAL - Enter paleo water depth information here! # # # # # # # # 
    # Import the data file (.csv)
    wd_csv = importdataset("Input/McCoyShort/McCoy_seq_Short2.csv", ',')
    # Obtain paleo water depth info from the data file
    wd_nLayers = length(wd_csv["Thickness"])
    wd = NewWaterDepth(wd_nLayers)
    wd.DepthID    = wd_csv["Type"]
    wd.Thickness .= wd_csv["Thickness"] 
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #


    # # # # # # # # # # Configure MC model here! # # # # # # # # # #
    # Number of MC simulations
    nsims = 5000
    # Resolution for model horizons (in km)
    res = 0.01
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    
    #SET FILEPATH#
    Path = "Output/Short_TCU6-Pioche/"
    # Run the decompaction and backstripping MC model with water depth
    
    @time (Sₜ, Sμ, Sσ, model_strat_heights) = DecompactBackstrip(strat, wd, nsims, res)

    # Code for storing and reading decompaction + backstripping results - will be useful when testing the age-depth modeling part of the model
    # Store results
    using DelimitedFiles
    writedlm(Path*"Mc_highres.csv", Sₜ, ",")
    writedlm(Path*"Mc_mu_highres.csv", Sμ)
    writedlm(Path*"Mc_sigma_highres.csv", Sσ)
    writedlm(Path*"Mc_highres.txt", Sₜ)
    # Read results
    Sₜ = readdlm(Path*"Mc_highres.csv", ',', Float64)
    Sμ = readdlm(Path*"Mc_mu_highres.csv", ',', Float64)
    Sσ = readdlm(Path*"Mc_sigma_highres.csv", ',', Float64)
    

In [ ]:
 # Plot results - tectonic subsidence in comparison with present day stratigraphic heights
    p1 = plot(Sμ, alpha = 1, yflip = true, xflip = true, label = "Tectonic subsidence", color = "black")
    #plot!(p1, reverse((model_strat_heights)*1000), yflip = true, label = "Present-day thickness", color = "red")
plot!(p1, Sₜ[:,2:end], alpha = 0.01, label = "", yflip = true, color = "green", fg_color_legend=:white)
    savefig(p1, Path*"McCoy_Short_DecompactBackstrip_higherres.pdf")


display(p1)

In [ ]:
#Input the number of samples to be used in model, must match below.
    nSamples = 6 #SEE BELOW
# Make an instance of a ChronSection object for nSamples
    smpl = NewChronAgeData(nSamples)
    smpl.Name          = ("TC7carbs","Base-TC7diamict_base_Marinoan","Base-Yelland_end_Marinoan",
                          "Mid-Osceola_Shuram_chemoCorr","Pioche_base","Top_ElyLimestone") #Strat_height_name
    smpl.Age           = [651.7,640,635.3,570,507,315.2] # Strat_age, MEASURED
    smpl.Age_sigma     = [.6,2,1.1,3,3,0.4]/2 # 2-σ age uncertainties, divided by 2; chron.jl uses 1sigma uncertainties as input data
    smpl.Height[:]     = [-10159,-9760,-9660,-6910,-5350,0] # Strat height in generalized Trout Creek-McCoy Creek Group section
    smpl.Height_sigma[:]  = [0,0,0,0,0,0] # Strat height uncertainty: initial try at 10m minimum for generalized section. 
    smpl.Age_Sidedness[:] = [-1,0,0,0,0,1] # Sidedness (zeros by default: geochron constraints are two-sided). Use -1 for a maximum age and +1 for a minimum age, 0 for two-sided

 


 

    # # # # # # # # # # Enter thermal subsidence parameter priors here! # # # # # # # # # #
    # Enter initial guesses for the beta factor and thermal subsidence onset age and their uncertainties
    Beta = 5.5
    Beta_sigma = .75
    T0 = 645
    T0_sigma = 10

    therm = NewThermalSubsidenceParameters()
    therm.Param = [Beta, T0]
    therm.Sigma = [Beta_sigma, T0_sigma]
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #


In [ ]:
# # # # # # # # # # Configure MCMC model here! # # # # # # # # # #
    # Configure the stratigraphic MCMC model
    config = NewStratAgeModelConfiguration()
    # If in doubt, you can probably leave these parameters as-is
    config.resolution = res*1000 # Same units as sample height. Smaller is slower!
    config.bounding = 1.0 # how far away do we place runaway bounds, as a fraction of total section height. Larger is slower.
    (bottom, top) = extrema(smpl.Height)
    npoints_approx = round(Int,length(bottom:config.resolution:top) * (1 + 2*config.bounding))
    config.nsteps = 6000 # Number of steps to run in distribution MCMC 
    config.burnin = 1000*npoints_approx # Number to discard 
    config.sieve = round(Int,npoints_approx) # Record one out of every nsieve steps
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    # Let's see if adding just a little bit of systematic error on tectonic subsidence solves the ll = -Inf problem
    # Currently the systematic error is set to be = 1% of the sigma for the tectonic subsidence of the first (oldest) layer
    Sσ_corr = Sσ .+ ((Sμ[end-1]-Sμ[end])/100)


        #Run the model
        (subsmdl, agedist, lldist, beta_t0dist, lldist_burnin) = SubsidenceStratMetropolis(smpl, config, therm, model_strat_heights, Sμ, Sσ, Beta_sigma/10, T0_sigma/10)

       

In [ ]:
 # Plot 1: Age-depth model (mean and 95% confidence interval for both model and data)
        hdl = plot([subsmdl.Age_025CI; reverse(subsmdl.Age_975CI)],[subsmdl.Height; reverse(subsmdl.Height)], fill=(round(Int,minimum(subsmdl.Height)),0.5,:blue), label="model")
        plot!(hdl, subsmdl.Age, subsmdl.Height, linecolor=:blue, label="", fg_color_legend=:white) # Center line
        t = smpl.Age_Sidedness .== 0 # Two-sided constraints (plot in black)
        any(t) && plot!(hdl, smpl.Age[t], smpl.Height[t], xerror=(2*smpl.Age_sigma[t]),yerror=(2*smpl.Height_sigma[t]),label="data",seriestype=:scatter,color=:black)
        t = smpl.Age_Sidedness .== 1 # Minimum ages (plot in cyan)
        any(t) && plot!(hdl, smpl.Age[t], smpl.Height[t], xerror=(2*smpl.Age_sigma[t],zeros(count(t))),label="",seriestype=:scatter,color=:cyan,msc=:cyan)
        any(t) && zip(smpl.Age[t], smpl.Age[t].+nanmean(smpl.Age_sigma[t])*4, smpl.Height[t]) .|> x-> plot!([x[1],x[2]],[x[3],x[3]], arrow=true, label="", color=:cyan)
        t = smpl.Age_Sidedness .== -1 # Maximum ages (plot in orange)
        any(t) && plot!(hdl, smpl.Age[t], smpl.Height[t], xerror=(zeros(count(t)),2*smpl.Age_sigma[t]),label="",seriestype=:scatter,color=:orange,msc=:orange)
        any(t) && zip(smpl.Age[t], smpl.Age[t].-nanmean(smpl.Age_sigma[t])*4, smpl.Height[t]) .|> x-> plot!([x[1],x[2]],[x[3],x[3]], arrow=true, label="", color=:orange)
        plot!(hdl, xlabel="Age ($(smpl.Age_Unit))", ylabel="Height ($(smpl.Height_Unit))")
        #plot!(hdl, [smpl.Age[1], smpl.Height[1]],[smpl.Age[3], smpl.Height[3]])
        #savefig(hdl,"McCoy_Age_Height.pdf")
        display(hdl)

In [ ]:
 # Plot 2: Posterior distributions of beta
        post_beta = histogram(beta_t0dist[1,:], color="cornflowerblue", linecolor=nothing, alpha = 0.5, nbins=50)
        vline!([subsmdl.Beta_Median], linecolor = "black", linestyle=:dot, linewidth = 3)
        writedlm(Path*"McShort_betaT0.csv", beta_t0dist, ',')
        savefig(post_beta, Path*"McCoy_PosteriorBeta.pdf")
display(post_beta)
display(subsmdl.Beta_Median)

In [ ]:
  # Plot 3: Posterior distributions of t₀
        post_t0 = histogram(beta_t0dist[2,:], color="cornflowerblue", linecolor=nothing, alpha = 0.5, nbins=50)
        vline!([subsmdl.T0_Median], linecolor = "red", linestyle=:dot, linewidth = 3)
        savefig(post_t0, Path*"McCoy_PosteriorT0.pdf")
display(post_t0)
display(subsmdl.T0_Median)

In [ ]:
#Plot 4: Interpolate results for target horizons
height = [

-10
-20
-30
-40
-50
-60
-70
-80
-90
-100
-110
-120
-130
-140
-150
-160
-170
-180
-190
-200
-210
-220
-230
-240
-250
-260
-270
-280
-290
-300
-310
-320
-330
-340
-350
-360
-370
-380
-390
-400
-410
-420
-430
-440
-450
-460
-470
-480
-490
-500
-510
-520
-530
-540
-550
-560
-570
-580
-590
-600
-610
-620
-630
-640
-650
-660
-670
-680
-690
-700
-710
-720
-730
-740
-750
-760
-770
-780
-790
-800
-810
-820
-830
-840
-850
-860
-870
-880
-890
-900
-910
-920
-930
-940
-950
-960
-970
-980
-990
-1000
-1010
-1020
-1030
-1040
-1050
-1060
-1070
-1080
-1090
-1100
-1110
-1120
-1130
-1140
-1150
-1160
-1170
-1180
-1190
-1200
-1210
-1220
-1230
-1240
-1250
-1260
-1270
-1280
-1290
-1300
-1310
-1320
-1330
-1340
-1350
-1360
-1370
-1380
-1390
-1400
-1410
-1420
-1430
-1440
-1450
-1460
-1470
-1480
-1490
-1500
-1510
-1520
-1530
-1540
-1550
-1560
-1570
-1580
-1590
-1600
-1610
-1620
-1630
-1640
-1650
-1660
-1670
-1680
-1690
-1700
-1710
-1720
-1730
-1740
-1750
-1760
-1770
-1780
-1790
-1800
-1810
-1820
-1830
-1840
-1850
-1860
-1870
-1880
-1890
-1900
-1910
-1920
-1930
-1940
-1950
-1960
-1970
-1980
-1990
-2000
-2010
-2020
-2030
-2040
-2050
-2060
-2070
-2080
-2090
-2100
-2110
-2120
-2130
-2140
-2150
-2160
-2170
-2180
-2190
-2200
-2210
-2220
-2230
-2240
-2250
-2260
-2270
-2280
-2290
-2300
-2310
-2320
-2330
-2340
-2350
-2360
-2370
-2380
-2390
-2400
-2410
-2420
-2430
-2440
-2450
-2460
-2470
-2480
-2490
-2500
-2510
-2520
-2530
-2540
-2550
-2560
-2570
-2580
-2590
-2600
-2610
-2620
-2630
-2640
-2650
-2660
-2670
-2680
-2690
-2700
-2710
-2720
-2730
-2740
-2750
-2760
-2770
-2780
-2790
-2800
-2810
-2820
-2830
-2840
-2850
-2860
-2870
-2880
-2890
-2900
-2910
-2920
-2930
-2940
-2950
-2960
-2970
-2980
-2990
-3000
-3010
-3020
-3030
-3040
-3050
-3060
-3070
-3080
-3090
-3100
-3110
-3120
-3130
-3140
-3150
-3160
-3170
-3180
-3190
-3200
-3210
-3220
-3230
-3240
-3250
-3260
-3270
-3280
-3290
-3300
-3310
-3320
-3330
-3340
-3350
-3360
-3370
-3380
-3390
-3400
-3410
-3420
-3430
-3440
-3450
-3460
-3470
-3480
-3490
-3500
-3510
-3520
-3530
-3540
-3550
-3560
-3570
-3580
-3590
-3600
-3610
-3620
-3630
-3640
-3650
-3660
-3670
-3680
-3690
-3700
-3710
-3720
-3730
-3740
-3750
-3760
-3770
-3780
-3790
-3800
-3810
-3820
-3830
-3840
-3850
-3860
-3870
-3880
-3890
-3900
-3910
-3920
-3930
-3940
-3950
-3960
-3970
-3980
-3990
-4000
-4010
-4020
-4030
-4040
-4050
-4060
-4070
-4080
-4090
-4100
-4110
-4120
-4130
-4140
-4150
-4160
-4170
-4180
-4190
-4200
-4210
-4220
-4230
-4240
-4250
-4260
-4270
-4280
-4290
-4300
-4310
-4320
-4330
-4340
-4350
-4360
-4370
-4380
-4390
-4400
-4410
-4420
-4430
-4440
-4450
-4460
-4470
-4480
-4490
-4500
-4510
-4520
-4530
-4540
-4550
-4560
-4570
-4580
-4590
-4600
-4610
-4620
-4630
-4640
-4650
-4660
-4670
-4680
-4690
-4700
-4710
-4720
-4730
-4740
-4750
-4760
-4770
-4780
-4790
-4800
-4810
-4820
-4830
-4840
-4850
-4860
-4870
-4880
-4890
-4900
-4910
-4920
-4930
-4940
-4950
-4960
-4970
-4980
-4990
-5000
-5010
-5020
-5030
-5040
-5050
-5060
-5070
-5080
-5090
-5100
-5110
-5120
-5130
-5140
-5150
-5160
-5170
-5180
-5190
-5200
-5210
-5220
-5230
-5240
-5250
-5260
-5270
-5280
-5290
-5300
-5310
-5320
-5330
-5340
-5350
-5360
-5370
-5380
-5390
-5400
-5410
-5420
-5430
-5440
-5450
-5460
-5470
-5480
-5490
-5500
-5510
-5520
-5530
-5540
-5550
-5560
-5570
-5580
-5590
-5600
-5610
-5620
-5630
-5640
-5650
-5660
-5670
-5680
-5690
-5700
-5710
-5720
-5730
-5740
-5750
-5760
-5770
-5780
-5790
-5800
-5810
-5820
-5830
-5840
-5850
-5860
-5870
-5880
-5890
-5900
-5910
-5920
-5930
-5940
-5950
-5960
-5970
-5980
-5990
-6000
-6010
-6020
-6030
-6040
-6050
-6060
-6070
-6080
-6090
-6100
-6110
-6120
-6130
-6140
-6150
-6160
-6170
-6180
-6190
-6200
-6210
-6220
-6230
-6240
-6250
-6260
-6270
-6280
-6290
-6300
-6310
-6320
-6330
-6340
-6350
-6360
-6370
-6380
-6390
-6400
-6410
-6420
-6430
-6440
-6450
-6460
-6470
-6480
-6490
-6500
-6510
-6520
-6530
-6540
-6550
-6560
-6570
-6580
-6590
-6600
-6610
-6620
-6630
-6640
-6650
-6660
-6670
-6680
-6690
-6700
-6710
-6720
-6730
-6740
-6750
-6760
-6770
-6780
-6790
-6800
-6810
-6820
-6830
-6840
-6850
-6860
-6870
-6880
-6890
-6900
-6910
-6920
-6930
-6940
-6950
-6960
-6970
-6980
-6990
-7000
-7010
-7020
-7030
-7040
-7050
-7060
-7070
-7080
-7090
-7100
-7110
-7120
-7130
-7140
-7150
-7160
-7170
-7180
-7190
-7200
-7210
-7220
-7230
-7240
-7250
-7260
-7270
-7280
-7290
-7300
-7310
-7320
-7330
-7340
-7350
-7360
-7370
-7380
-7390
-7400
-7410
-7420
-7430
-7440
-7450
-7460
-7470
-7480
-7490
-7500
-7510
-7520
-7530
-7540
-7550
-7560
-7570
-7580
-7590
-7600
-7610
-7620
-7630
-7640
-7650
-7660
-7670
-7680
-7690
-7700
-7710
-7720
-7730
-7740
-7750
-7760
-7770
-7780
-7790
-7800
-7810
-7820
-7830
-7840
-7850
-7860
-7870
-7880
-7890
-7900
-7910
-7920
-7930
-7940
-7950
-7960
-7970
-7980
-7990
-8000
-8010
-8020
-8030
-8040
-8050
-8060
-8070
-8080
-8090
-8100
-8110
-8120
-8130
-8140
-8150
-8160
-8170
-8180
-8190
-8200
-8210
-8220
-8230
-8240
-8250
-8260
-8270
-8280
-8290
-8300
-8310
-8320
-8330
-8340
-8350
-8360
-8370
-8380
-8390
-8400
-8410
-8420
-8430
-8440
-8450
-8460
-8470
-8480
-8490
-8500
-8510
-8520
-8530
-8540
-8550
-8560
-8570
-8580
-8590
-8600
-8610
-8620
-8630
-8640
-8650
-8660
-8670
-8680
-8690
-8700
-8710
-8720
-8730
-8740
-8750
-8760
-8770
-8780
-8790
-8800
-8810
-8820
-8830
-8840
-8850
-8860
-8870
-8880
-8890
-8900
-8910
-8920
-8930
-8940
-8950
-8960
-8970
-8980
-8990
-9000
-9010
-9020
-9030
-9040
-9050
-9060
-9070
-9080
-9090
-9100
-9110
-9120
-9130
-9140
-9150
-9160
-9170
-9180
-9190
-9200
-9210
-9220
-9230
-9240
-9250
-9260
-9270
-9280
-9290
-9300
-9310
-9320
-9330
-9340
-9350
-9360
-9370
-9380
-9390
-9400
-9410
-9420
-9430
-9440
-9450
-9460
-9470
-9480
-9490
-9500
-9510
-9520
-9530
-9540
-9550
-9560
-9570
-9580
-9590
-9600
-9610
-9620
-9630
-9640
-9650
-9660
-9670
-9680
-9690
-9700
-9710
-9720
-9730
-9740
-9750
-9760
-9770
-9780
-9790
-9800
-9810
-9820
-9830
-9840
-9850
-9860
-9870
-9880
-9890
-9900
-9910
-9920
-9930
-9940
-9950
-9960
-9970
-9980
-9990
-10000
-10010
-10020
-10030
-10040
-10050
-10060
-10070
-10080
-10090
-10100
-10110
-10120
-10130
-10140
-10150
-10160

]

single_age_at_height = linterp1s(subsmdl.Height,subsmdl.Age,height)
writedlm(Path*"Age_at_height_McCoy.csv", single_age_at_height, ',')

In [ ]:
 # Plot results - tectonic subsidence in comparison with present day stratigraphic heights
    age=single_age_at_height
    p = plot(age, Sμ, yflip=true, xflip=true, color="green")
    #plot!(p1, reverse((model_strat_heights)*1000), yflip = true, label = "Present-day thickness", color = "red")
plot!(p,(single_age_at_height),Sₜ[:,2:end], alpha = 0.01, label = "", yflip = true, xflip=true, color = "green", fg_color_legend=:white)
    
    display(p)

In [ ]:
# Match model to observed
        p6 = plot(age, Sμ, yflip=true, xflip=true, color="navy",order=3,alpha=.1)
      
        τ = 65
        E₀ = 3200
        for i = 1:3000
            xrange = 300:((beta_t0dist[2,i]-300)/1000):(beta_t0dist[2,i])
            tec_subs = (E₀*beta_t0dist[1,i]/pi)*sin(pi/beta_t0dist[1,i]).*(1 .-exp.(-(beta_t0dist[2,i] .-xrange)./τ))
            plot!(p6, xrange, tec_subs, alpha = 0.01,linewidth=.1, color = "red", legend = false,xlim=(400,660),order=4)
        end
        
plot!(p6,(single_age_at_height),Sₜ[:,2:end], linewidth=.1, alpha = 0.03, label = "", yflip = true, xflip=true, color = "navy", fg_color_legend=:white, order=3)
        plot(p6,age, Sμ, yflip=true, xflip=true, color="navy",alpha=.3)
vline!(p6,[subsmdl.T0_Median], linecolor = "red",linestyle=:dash, linewidth = 1,alpha=.5,order=1)

 #plot Marinoan
vspan!(p6,[635,640]; color = "navy",alpha=.15,order=1)
#plot Gaskiers
vspan!(p6,[579.24,580.90]; color = "navy",alpha=.15,order=1)
#plot Avalonia glacial       
vline!(p6,[565], linecolor = "navy", linewidth = 1.5,alpha=.15,order=1)
#plot Hirnantian glacial
vline!(p6,[443], linecolor = "navy", linewidth = 1.5,alpha=.15,order=1)


#plot Shuram
vspan!(p6,[565.24,572]; color = "gray",alpha=.15,order=1)

    #plot Sauk
vspan!(p6,[470,510]; color = "teal",alpha=.15,order=1)




savefig(p6, Path*"McCoy_ThermSubs_Test.pdf")
display(p6)